# Preprocessing

In [8]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion DNSMOS

In [10]:
import numpy as np
import onnxruntime as ort

class DNSMOS:
    def __init__(self, model_path="models/sig_bak_ovr.onnx"):
        """
        Initialisiert das DNSMOS-Modell für SIGMOS, BAKMOS und OVRMOS.
        Quelle: Microsoft Research, Deep Noise Suppression Mean Opinion Score.
        """
        self.model = ort.InferenceSession(model_path)

    def calculate_dnsmos(self, audio_signal):
        """
        Berechnet DNSMOS für ein Audio-Signal.

        Args:
            audio_signal (np.array): Das vorverarbeitete Audio-Signal (Mono, 16 kHz, Normalisiert).

        Returns:
            dict: DNSMOS-Scores (SIGMOS, BAKMOS, OVRMOS).
        """
        # Sicherstellen, dass das Signal die richtige Form hat
        audio_signal = np.expand_dims(audio_signal, axis=0).astype(np.float32)

        # ONNX-Modelleingabe & -Ausgabe abrufen
        input_name = self.model.get_inputs()[0].name
        output_names = [out.name for out in self.model.get_outputs()]
        
        # Modellvorhersage
        result = self.model.run(output_names, {input_name: audio_signal})

        # DNSMOS-Ergebnisse zurückgeben
        return {
            "SIGMOS": result[0][0],  # Speech Quality
            "BAKMOS": result[1][0],  # Background Noise Quality
            "OVRMOS": result[2][0]   # Overall Quality
        }

# Test

In [14]:
import os

# Verzeichnis mit den Audiodateien
audio_dir = "../audio_files/"  # <-- Hier den korrekten Pfad anpassen

# Alle Audiodateien im Verzeichnis durchsuchen
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]

# DNSMOS-Modell initialisieren
dnsmos_model = DNSMOS(model_path="models/sig_bak_ovr.onnx")  # Falls woanders gespeichert, Pfad anpassen

# Durch alle Audiodateien iterieren und DNSMOS berechnen
for file_name in audio_files:
    file_path = os.path.join(audio_dir, file_name)

    # Audio vorverarbeiten (Resampling, Mono, Normalisierung)
    audio_signal = preprocess_audio(file_path)

    # DNSMOS-Scores berechnen
    dnsmos_scores = dnsmos_model.calculate_dnsmos(audio_signal)

    # Ergebnisse ausgeben
    if dnsmos_scores:
        print(f"DNSMOS ({file_name}): SIGMOS={dnsmos_scores['SIGMOS']:.2f}, "
              f"BAKMOS={dnsmos_scores['BAKMOS']:.2f}, OVRMOS={dnsmos_scores['OVRMOS']:.2f}")
    else:
        print(f"Fehler: Keine DNSMOS-Werte für {file_name} berechnet.")

NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from models/sig_bak_ovr.onnx failed:Load model models/sig_bak_ovr.onnx failed. File doesn't exist